# Matching records extracted from text to BLL01 Incunabula

Using VM's ss where she matched our extracted records with BLL01

In [12]:
import sys
if "../" not in sys.path:
    sys.path.append("../")
import glob
import pandas as pd

from src.data.reimport_utils import reconstruct_xmls

In [10]:
entry_paths = sorted(glob.glob("../data/processed/*/catalogue_entries_v1.4.csv"), key=lambda x: int(x.split("BMC_")[1].split("\\")[0]))

In [20]:
test_df = pd.read_csv(entry_paths[0], converters={"xmls": reconstruct_xmls}, encoding="utf8")

In [18]:
test_df.head()

,xmls,xml_start_line,vol_entry_num,shelfmark,entry,title,entry_text,word_locations
0,"[J_2704_aa_30_1_0038_2, J_2704_aa_30_1_0039_2]","[62, 72]",0,IB. 14,"['BLOCKBOOKS', '(Arranged alphabetically.)', '...","[59, 60, 61, 62]",BLOCKBOOKS\n(Arranged alphabetically.)\nLeaf 2...,"[[[(1283, 1008), (2301, 1068)]], [[(1393, 1156..."
1,[J_2704_aa_30_1_0039_2],[43],1,IB. 17,"['ARS MEMORANDI. Ars memorandi per', 'figuras ...","[130, 131]",ARS MEMORANDI. Ars memorandi per\nfiguras euan...,"[[[(765, 1127), (907, 1187)], [(972, 1119), (1..."
2,"[J_2704_aa_30_1_0039_2, J_2704_aa_30_1_0040_2]","[94, 108]",2,IB. 23,"['ARS MORIENDI.', '[District of the Rhine, c. ...","[173, 174]","ARS MORIENDI.\n[District of the Rhine, c. 1465...","[[[(795, 3566), (937, 3626)], [(1010, 3559), (..."
3,[J_2704_aa_30_1_0040_2],[30],3,IA. 24,"['ARS MORIENDI. German text.', '[Germany :] Ha...","[280, 281]",ARS MORIENDI. German text.\n[Germany :] Hans S...,"[[[(300, 1376), (435, 1436)], [(512, 1372), (9..."
4,[J_2704_aa_30_1_0040_2],[25],4,IB. 21,"['ARS MORIENDI. A compendium, in Ger¬', 'man.'...","[310, 311, 312]","ARS MORIENDI. A compendium, in Ger¬\nman.\n[Ge...","[[[(317, 3396), (450, 3456)], [(527, 3404), (9..."


In [21]:
entry_dfs = [pd.read_csv(x, converters={"xmls": reconstruct_xmls}, encoding="utf8") for x in entry_paths]

In [125]:
ocr_records_df = pd.concat(entry_dfs)[["xmls", "vol_entry_num", "shelfmark", "entry_text"]].reset_index(drop=True)
ocr_records_df["entry_text_spaces"] = ocr_records_df["entry_text"].str.replace("\n", " ")
# ocr_records_df = pd.read_csv("..\\data\\processed\\all_volume_catalogue_entries_v1.0.csv", encoding='latin-1', index_col="Column1")
bll01_index_df = pd.read_csv("..\\data\\processed\\bll01_index.csv", encoding='utf-8')
bll01_index_df.rename(columns={'British Library shelfmark (852 $j)': "bll01_shelfmark", 'Record IDs (001)': "record_id"}, inplace=True)

In [126]:
ocr_records_df

,xmls,vol_entry_num,shelfmark,entry_text,entry_text_spaces
0,"[J_2704_aa_30_1_0038_2, J_2704_aa_30_1_0039_2]",0,IB. 14,BLOCKBOOKS\n(Arranged alphabetically.)\nLeaf 2...,BLOCKBOOKS (Arranged alphabetically.) Leaf 21 ...
1,[J_2704_aa_30_1_0039_2],1,IB. 17,ARS MEMORANDI. Ars memorandi per\nfiguras euan...,ARS MEMORANDI. Ars memorandi per figuras euang...
2,"[J_2704_aa_30_1_0039_2, J_2704_aa_30_1_0040_2]",2,IB. 23,"ARS MORIENDI.\n[District of the Rhine, c. 1465...","ARS MORIENDI. [District of the Rhine, c. 1465?..."
3,[J_2704_aa_30_1_0040_2],3,IA. 24,ARS MORIENDI. German text.\n[Germany :] Hans S...,ARS MORIENDI. German text. [Germany :] Hans Sp...
4,[J_2704_aa_30_1_0040_2],4,IB. 21,"ARS MORIENDI. A compendium, in Ger¬\nman.\n[Ge...","ARS MORIENDI. A compendium, in Ger¬ man. [Germ..."
...,...,...,...,...,...
8769,"[J_2740_aa_30_10_0159_2, J_2740_aa_30_10_0160_4]",172,IA. 54323,RESPONSORIA OFFICII DEFUNCTO¬\nRUM.\n1500.\n1a...,RESPONSORIA OFFICII DEFUNCTO¬ RUM. 1500. 1a P ...
8770,"[J_2740_aa_30_10_0160_4, J_2740_aa_30_10_0161_...",173,IC. 56659,Vita\nLUDOLPHUS DE SAXONIA.\nChristi. [In the ...,Vita LUDOLPHUS DE SAXONIA. Christi. [In the Po...
8771,[J_2740_aa_30_10_0162_4],174,IA. 56660,VALENTIM FERNANDES ALONE\nVOTIVALE. Votiuale m...,VALENTIM FERNANDES ALONE VOTIVALE. Votiuale mi...
8772,"[J_2740_aa_30_10_0162_4, J_2740_aa_30_10_0163_4]",175,IA. 56710,"ZACUTUS, ABRAHAM. Almanach\nper¬\npetuum coele...","ZACUTUS, ABRAHAM. Almanach per¬ petuum coelest..."


In [42]:
matched_records_df = pd.read_csv("..\\data\\processed\\bll_matching_entries.csv", encoding='latin-1', index_col="index to column1").sort_index().iloc[:-1]

In [47]:
matched_records_df.index.astype(int)

Index([   0,    1,    2,    3,    4,    5,    6,    7,    8,    9,
       ...
       8804, 8805, 8806, 8807, 8808, 8809, 8810, 8811, 8812, 8813],
      dtype='int32', name='index to column1', length=8814)

In [53]:
ocr_records_df.loc[:,["vol_entry_num", "xmls"]]

,vol_entry_num,xmls
0,0,"[J_2704_aa_30_1_0038_2, J_2704_aa_30_1_0039_2]"
1,1,[J_2704_aa_30_1_0039_2]
2,2,"[J_2704_aa_30_1_0039_2, J_2704_aa_30_1_0040_2]"
3,3,[J_2704_aa_30_1_0040_2]
4,4,[J_2704_aa_30_1_0040_2]
...,...,...
8769,172,"[J_2740_aa_30_10_0159_2, J_2740_aa_30_10_0160_4]"
8770,173,"[J_2740_aa_30_10_0160_4, J_2740_aa_30_10_0161_..."
8771,174,[J_2740_aa_30_10_0162_4]
8772,175,"[J_2740_aa_30_10_0162_4, J_2740_aa_30_10_0163_4]"


In [56]:
# matched_records_df = matched_records_df.set_index(matched_records_df.index.astype(int)).rename(columns={"shelfmark": "raw_shelfmark", "normalized shelfmark": "norm_shelfmark"})
# matched_records_df = matched_records_df.join(ocr_records_df.loc[:,["vol_entry_num", "xmls"]], how="left", rsuffix="_ocr")
# matched_records_df.insert(loc=3, column="vol_entry_num", value=matched_records_df["vol_entry_num"] + 1)
# matched_records_df.drop(columns="volume_entry_num", inplace=True)

In [55]:
matched_records_df

,xml,raw_shelfmark,norm_shelfmark,entry_text,Shelfmark 1,Record IDs for shelfmark 1,Shelfmark 2,Record IDs for shelfmark 2,Shelfmark 3,Record IDs for shelfmark 3,...,Shelfmark 16,Record IDs for shelfmark 16,Shelfmark 17,Record IDs for shelfmark 17,Shelfmark 18,Record IDs for shelfmark 18,Shelfmark 19,Record IDs for shelfmark 19,vol_entry_num,xmls
index to column1,,,,,,,,,,,,,,,,,,,,,
0,J_2704_aa_30_1_0038_2,IB. 41,IB. 41,BLOCKBOOKS (Arranged alphabetically.) Leaf 21 ...,IB. 41,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,"[J_2704_aa_30_1_0038_2, J_2704_aa_30_1_0039_2]"
1,J_2704_aa_30_1_0038_2,IB. 14,IB. 14,"King George III's copy (C. 9. c. 1). [Germany,...",C. 9. c. 1,NaN,IB. 16,344420,IB. 14,344421,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,[J_2704_aa_30_1_0039_2]
2,J_2704_aa_30_1_0039_2,IB. 17,IB. 17,"Bought in June, 1872. ARS MEMORANDI. Ars memor...",IB. 17,122790,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,"[J_2704_aa_30_1_0039_2, J_2704_aa_30_1_0040_2]"
3,J_2704_aa_30_1_0039_2,IB. 23,IB. 23,"Bought in September, 1854. ARS MORIENDI. [Dist...",IA. 19,122829,132. l. 28,NaN,IB. 18,122828,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,[J_2704_aa_30_1_0040_2]
4,J_2704_aa_30_1_0040_2,IA. 24ARS,IA. 24,"Bought in February, 1846. ARS MORIENDI. German...",IA. 24,122851,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,[J_2704_aa_30_1_0040_2]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8809,J_2704_aa_30_9_0150_4,IA. 47395,IA. 47395,FREESKA LANDRIUCHT. Not after 1488? 12. tHer e...,IA. 47395,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8810,J_2704_aa_30_9_0152_4,IB. 47504,IB. 47504,"BERCHORIUS, PETRUS. Liber Bibliae JACOBUS DE V...",IB. 47501,280118,IB. 47503,1839331,IB. 47504,1839332,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8811,J_2704_aa_30_9_0153_4,IA. 47935,IA. 47935,"BOORT, HENRICUS. Fasciculus morum. Undated. 12...",IA. 47935,3436823,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [57]:
ocr_records_df.shape, bll01_index_df.shape, matched_records_df.shape

((8774, 9), (11392, 7), (8814, 44))

In [ ]:
ocr_records_df.head()

Going to stop using `norm_shelfmark` from VM's ss. This is `normalized shelfmark` in `bll_matchin_entries.csv`. Going to match instead on `shelfmark` from the combined `catalogue_entries_v1.4.csv`, which is the version produced using the latest entry extraction algorithm.

In [26]:
ocr_shelfmarks = ocr_records_df["shelfmark"].str.strip()
# ocr_shelfmarks = matched_records_df["norm_shelfmark"].str.strip()

In [ ]:
bll01_index_df["bll01_shelfmark"] = bll01_index_df["bll01_shelfmark"].str.strip()

In [ ]:
bll_ocr_df = bll01_index_df.merge(ocr_shelfmarks.drop_duplicates(), left_on="bll01_shelfmark", right_on="norm_shelfmark", how="left").rename(columns={"normalized_shelfmark": "normalized_ocr_shelfmark"})
# bll_ocr_df = bll01_index_df.merge(ocr_shelfmarks, left_on="bll01_shelfmark", right_on="normalized_shelfmark", how="right").rename(columns={"normalized_shelfmark": "normalized_ocr_shelfmark"})

In [ ]:
bll01_index_df

In [ ]:
ocr_shelfmarks

In [ ]:
bll_ocr_df

In [ ]:
bll_ocr_df.shape

In [ ]:
# ocr_bll_df.to_csv("..\\data\\processed\\ocr_bll.csv", index=False)
# bll_ocr_df.to_csv("..\\data\\processed\\bll_ocr.csv", index=False)

### Creating Tkb page col lookup

Adding a column that converts the xml name to a page number in the Tkb collection

In [ ]:
matched_records_df.head()

In [48]:
# all the xml ids present on tkb - if the code didn't find a shelfmark in an xml it won't be represented in matched_records
# use the complete list to convert our incomplete list into page numbers on tkb
with open("..\\data\\interim\\all_xml_ids.txt", "r") as f:
    xml_ids = [x.strip("\n") for x in f.readlines()]

In [49]:
xml_ids[:5]

['J_2704_aa_30_1_0038_2',
 'J_2704_aa_30_1_0039_2',
 'J_2704_aa_30_1_0040_2',
 'J_2704_aa_30_1_0041_2',
 'J_2704_aa_30_1_0042_2']

In [65]:
# check all our xmls are present in the complete list
set(ocr_records_df["xmls"].sum()) <= set(xml_ids)
# matched_records_df["xml"].apply(lambda x: x in xml_ids).all()

True

In [ ]:
all_xml_df = pd.DataFrame({"xml":xml_ids})

all_xml_df["vol"] = all_xml_df["xml"].str.split("_").apply(lambda x: int(x[4]))
all_xml_df["raw_tkb_page"] = all_xml_df["xml"].str.split("_").apply(lambda x: int(x[5]))
all_xml_df["num_cols"] = all_xml_df["xml"].str.split("_").apply(lambda x: int(x[6]))
all_xml_df.sort_values(by=["vol", "num_cols", "raw_tkb_page"], inplace=True)
all_xml_df["tkb_page"] = all_xml_df.groupby(by=["vol", "num_cols"])["raw_tkb_page"].transform(lambda x: (x.diff() > 0).cumsum() + 1)
all_xml_df = all_xml_df.set_index("xml")

In [ ]:
all_xml_df.query("vol == 9 & num_cols == 4").head()

In [ ]:
# all_xml_df.to_csv("..\\data\\processed\\xml_tkb_conversion_table.csv")

### Using Tkb page col lookup

In [66]:
all_xml_df = pd.read_csv("..\\data\\processed\\xml_tkb_conversion_table.csv", index_col=0)

In [67]:
bll01_index_df.columns

Index(['bll01_shelfmark', 'record_id', 'c_sm', 'i_sm', 'g_sm', 'uncaptured_sm',
       'needs_capturing'],
      dtype='object')

In [78]:
# ocr_records_df["first_xml"] = ocr_records_df["xmls"].transform(lambda x: x[0])
# ocr_records_df["last_xml"] = ocr_records_df["xmls"].transform(lambda x: x[-1])

In [121]:
ocr_records_df

,xmls,vol_entry_num,shelfmark,entry_text
0,"[J_2704_aa_30_1_0038_2, J_2704_aa_30_1_0039_2]",0,IB. 14,BLOCKBOOKS\n(Arranged alphabetically.)\nLeaf 2...
1,[J_2704_aa_30_1_0039_2],1,IB. 17,ARS MEMORANDI. Ars memorandi per\nfiguras euan...
2,"[J_2704_aa_30_1_0039_2, J_2704_aa_30_1_0040_2]",2,IB. 23,"ARS MORIENDI.\n[District of the Rhine, c. 1465..."
3,[J_2704_aa_30_1_0040_2],3,IA. 24,ARS MORIENDI. German text.\n[Germany :] Hans S...
4,[J_2704_aa_30_1_0040_2],4,IB. 21,"ARS MORIENDI. A compendium, in Ger¬\nman.\n[Ge..."
...,...,...,...,...
8769,"[J_2740_aa_30_10_0159_2, J_2740_aa_30_10_0160_4]",172,IA. 54323,RESPONSORIA OFFICII DEFUNCTO¬\nRUM.\n1500.\n1a...
8770,"[J_2740_aa_30_10_0160_4, J_2740_aa_30_10_0161_...",173,IC. 56659,Vita\nLUDOLPHUS DE SAXONIA.\nChristi. [In the ...
8771,[J_2740_aa_30_10_0162_4],174,IA. 56660,VALENTIM FERNANDES ALONE\nVOTIVALE. Votiuale m...
8772,"[J_2740_aa_30_10_0162_4, J_2740_aa_30_10_0163_4]",175,IA. 56710,"ZACUTUS, ABRAHAM. Almanach\nper¬\npetuum coele..."


In [137]:
vol = pd.merge(ocr_records_df["xmls"].apply(lambda x: x[0]), all_xml_df.drop(columns="raw_tkb_page"), how="left", left_on="xmls", right_on="xml")["vol"]
xmls_long = pd.merge(ocr_records_df["xmls"].explode().reset_index(), all_xml_df.drop(columns="raw_tkb_page"), how="left", left_on="xmls", right_on="xml")
xmls_long["col_pages"] = xmls_long["num_cols"].astype(str) + "-" + xmls_long["tkb_page"].astype(str) + ", "
col_pages = xmls_long.groupby(by="index")["col_pages"].sum().str.strip(" ")

In [138]:
ocr_records_df["vol"] = vol
ocr_records_df["col_pages"] = col_pages

In [139]:
ocr_records_df

,xmls,vol_entry_num,shelfmark,entry_text,entry_text_spaces,vol,col_pages
0,"[J_2704_aa_30_1_0038_2, J_2704_aa_30_1_0039_2]",0,IB. 14,BLOCKBOOKS\n(Arranged alphabetically.)\nLeaf 2...,BLOCKBOOKS (Arranged alphabetically.) Leaf 21 ...,1,"2-1, 2-2,"
1,[J_2704_aa_30_1_0039_2],1,IB. 17,ARS MEMORANDI. Ars memorandi per\nfiguras euan...,ARS MEMORANDI. Ars memorandi per figuras euang...,1,"2-2,"
2,"[J_2704_aa_30_1_0039_2, J_2704_aa_30_1_0040_2]",2,IB. 23,"ARS MORIENDI.\n[District of the Rhine, c. 1465...","ARS MORIENDI. [District of the Rhine, c. 1465?...",1,"2-2, 2-3,"
3,[J_2704_aa_30_1_0040_2],3,IA. 24,ARS MORIENDI. German text.\n[Germany :] Hans S...,ARS MORIENDI. German text. [Germany :] Hans Sp...,1,"2-3,"
4,[J_2704_aa_30_1_0040_2],4,IB. 21,"ARS MORIENDI. A compendium, in Ger¬\nman.\n[Ge...","ARS MORIENDI. A compendium, in Ger¬ man. [Germ...",1,"2-3,"
...,...,...,...,...,...,...,...
8769,"[J_2740_aa_30_10_0159_2, J_2740_aa_30_10_0160_4]",172,IA. 54323,RESPONSORIA OFFICII DEFUNCTO¬\nRUM.\n1500.\n1a...,RESPONSORIA OFFICII DEFUNCTO¬ RUM. 1500. 1a P ...,10,"2-34, 4-48,"
8770,"[J_2740_aa_30_10_0160_4, J_2740_aa_30_10_0161_...",173,IC. 56659,Vita\nLUDOLPHUS DE SAXONIA.\nChristi. [In the ...,Vita LUDOLPHUS DE SAXONIA. Christi. [In the Po...,10,"4-48, 2-35, 4-49,"
8771,[J_2740_aa_30_10_0162_4],174,IA. 56660,VALENTIM FERNANDES ALONE\nVOTIVALE. Votiuale m...,VALENTIM FERNANDES ALONE VOTIVALE. Votiuale mi...,10,"4-49,"
8772,"[J_2740_aa_30_10_0162_4, J_2740_aa_30_10_0163_4]",175,IA. 56710,"ZACUTUS, ABRAHAM. Almanach\nper¬\npetuum coele...","ZACUTUS, ABRAHAM. Almanach per¬ petuum coelest...",10,"4-49, 4-50,"


In [140]:
bll01_index_df

,bll01_shelfmark,record_id,c_sm,i_sm,g_sm,uncaptured_sm,needs_capturing
0,1010. c. 2,1877377,0,0,0,1,0
1,1073. h. 4,268186,0,0,0,1,0
2,11315. i. 32. (2. ),1722918,0,0,0,1,0
3,11352. bb. 10,412770,0,0,0,1,0
4,11405. bb. 43,38048,0,0,0,1,0
...,...,...,...,...,...,...,...
11387,OC IA. 22,16469823,0,1,0,0,0
11388,OC IA. 49865,18760162,0,1,0,0,0
11389,OC ORB. 40/284,3004341,0,0,0,1,0
11390,RoyalMS. 12. B. XXIV,003615926;002928154;004958758,0,0,0,1,0


In [141]:
matched_records_merged_df = ocr_records_df.merge(right=bll01_index_df, how="left", left_on="shelfmark", right_on="bll01_shelfmark")
matched_records_merged_df = matched_records_merged_df[[
    'xmls', 'vol', 'col_pages', 'vol_entry_num', 'shelfmark', 'bll01_shelfmark', 'record_id', 'uncaptured_sm', 'entry_text', "entry_text_spaces"
]]

In [142]:
matched_records_merged_df

,xmls,vol,col_pages,vol_entry_num,shelfmark,bll01_shelfmark,record_id,uncaptured_sm,entry_text,entry_text_spaces
0,"[J_2704_aa_30_1_0038_2, J_2704_aa_30_1_0039_2]",1,"2-1, 2-2,",0,IB. 14,IB. 14,344421,0.0,BLOCKBOOKS\n(Arranged alphabetically.)\nLeaf 2...,BLOCKBOOKS (Arranged alphabetically.) Leaf 21 ...
1,[J_2704_aa_30_1_0039_2],1,"2-2,",1,IB. 17,IB. 17,122790,0.0,ARS MEMORANDI. Ars memorandi per\nfiguras euan...,ARS MEMORANDI. Ars memorandi per figuras euang...
2,"[J_2704_aa_30_1_0039_2, J_2704_aa_30_1_0040_2]",1,"2-2, 2-3,",2,IB. 23,IB. 23,122830,0.0,"ARS MORIENDI.\n[District of the Rhine, c. 1465...","ARS MORIENDI. [District of the Rhine, c. 1465?..."
3,[J_2704_aa_30_1_0040_2],1,"2-3,",3,IA. 24,IA. 24,122851,0.0,ARS MORIENDI. German text.\n[Germany :] Hans S...,ARS MORIENDI. German text. [Germany :] Hans Sp...
4,[J_2704_aa_30_1_0040_2],1,"2-3,",4,IB. 21,IB. 21,122856,0.0,"ARS MORIENDI. A compendium, in Ger¬\nman.\n[Ge...","ARS MORIENDI. A compendium, in Ger¬ man. [Germ..."
...,...,...,...,...,...,...,...,...,...,...
8769,"[J_2740_aa_30_10_0159_2, J_2740_aa_30_10_0160_4]",10,"2-34, 4-48,",172,IA. 54323,NaN,NaN,NaN,RESPONSORIA OFFICII DEFUNCTO¬\nRUM.\n1500.\n1a...,RESPONSORIA OFFICII DEFUNCTO¬ RUM. 1500. 1a P ...
8770,"[J_2740_aa_30_10_0160_4, J_2740_aa_30_10_0161_...",10,"4-48, 2-35, 4-49,",173,IC. 56659,IC. 56659,2279663,0.0,Vita\nLUDOLPHUS DE SAXONIA.\nChristi. [In the ...,Vita LUDOLPHUS DE SAXONIA. Christi. [In the Po...
8771,[J_2740_aa_30_10_0162_4],10,"4-49,",174,IA. 56660,IA. 56660,2190396,0.0,VALENTIM FERNANDES ALONE\nVOTIVALE. Votiuale m...,VALENTIM FERNANDES ALONE VOTIVALE. Votiuale mi...
8772,"[J_2740_aa_30_10_0162_4, J_2740_aa_30_10_0163_4]",10,"4-49, 4-50,",175,IA. 56710,IA. 56710,4007659,0.0,"ZACUTUS, ABRAHAM. Almanach\nper¬\npetuum coele...","ZACUTUS, ABRAHAM. Almanach per¬ petuum coelest..."


### Export

In [143]:
matched_records_merged_df.to_csv("..\\data\\processed\\incu_ocr_bll_matching_v1.4.csv", encoding="utf-8-sig")

Then add these sheets manually
- BLL01 index
- Recording for missing shelfmarks and the reason for them being missing
- Any edits made to tkb as a result
- Explanation of the workflow

### Preserving VMs shelfmark extraction in compressed form

In [ ]:
compressed_shelfmark_rec_ids = matched_records_df.loc[:, "Shelfmark 1":].agg(
    lambda x: [[y, z] for y, z in zip(x[::2], x[1::2]) if type(y) == str], axis=1
).rename("shelfmarks_in_entry_text")

In [ ]:
compressed_shelfmark_rec_ids
# matched_records_tkb_page_df["vm_shelfmark_rec_id_matching"] = compressed_shelfmark_rec_ids

In [ ]:
# check we've as many shelfmarks in the compressed as non-compressed
counts = matched_records_df.loc[:, "Shelfmark 1":].agg(lambda x: x.dropna().shape[0])
counts.iloc[::2].sum(), len(compressed_shelfmark_rec_ids.sum())

In [ ]:
# check we've as many record ids in the compressed as the non-compressed
# not quite correct but then not really using this data anyway
counts.iloc[1::2].sum(), len([x[1] for x in compressed_shelfmark_rec_ids.sum() if type(x[1]) == str])

## Trialling